In [1]:
import syft as sy
hook = sy.TorchHook()

In [2]:
bob = sy.VirtualWorker(id='bob')
alice = sy.VirtualWorker(id='alice')

In [3]:
# making sure that bob/alice know about each other
bob.add_worker(alice)
alice.add_worker(bob)

In [4]:
# this is a local tensor
x = sy.FloatTensor([1,2,3,4])
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [5]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

FloatTensor[_PointerTensor - id:53153638636 owner:me loc:bob id@loc:20440673954]

In [6]:
x_ptr.child.original_pointer

True

In [7]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

FloatTensor[_PointerTensor - id:53153638636 owner:me loc:alice id@loc:3778386636]

In [8]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{20440673954: [_LocalTensor - id:20440673954 owner:bob]}

In [9]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{3778386636: [_PointerTensor - id:3778386636 owner:alice loc:bob id@loc:20440673954]}

In [10]:
# and we can use .get() to get x_ptr back from Alice
x_ptr = pointer_to_x_ptr.get()
x_ptr

FloatTensor[_PointerTensor - id:53153638636 owner:me loc:bob id@loc:20440673954]

In [11]:
# and then we can use x_ptr to get x back from Bob!
x = x_ptr.get()
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [12]:
bob._objects

{}

In [13]:
alice._objects

{}

In [14]:
p2p2x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [15]:
bob._objects

{54074173435: [_LocalTensor - id:54074173435 owner:bob],
 33575149550: [_LocalTensor - id:33575149550 owner:bob]}

In [16]:
alice._objects

{39840344196: [_PointerTensor - id:39840344196 owner:alice loc:bob id@loc:54074173435],
 2254345331: [_PointerTensor - id:2254345331 owner:alice loc:bob id@loc:33575149550]}

In [17]:
y.get().get()


  2
  4
  6
  8
 10
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [18]:
bob._objects

{54074173435: [_LocalTensor - id:54074173435 owner:bob]}

In [19]:
alice._objects

{39840344196: [_PointerTensor - id:39840344196 owner:alice loc:bob id@loc:54074173435]}

In [20]:
p2p2x.get().get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [21]:
bob._objects

{}

In [22]:
alice._objects

{}

In [23]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

In [24]:
bob._objects

{60953618507: [_LocalTensor - id:60953618507 owner:bob]}

In [25]:
alice._objects

{28322905380: [_PointerTensor - id:28322905380 owner:alice loc:bob id@loc:60953618507]}

In [26]:
x2 = x.end_get()

In [27]:
x2

FloatTensor[_PointerTensor - id:62517717240 owner:me loc:alice id@loc:28322905380]

In [28]:
bob._objects

{}

In [29]:
alice._objects

{28322905380: [_LocalTensor - id:28322905380 owner:alice]}

In [30]:
x2

FloatTensor[_PointerTensor - id:62517717240 owner:me loc:alice id@loc:28322905380]

In [31]:
x2.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [32]:

# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.Variable(sy.FloatTensor([1,2,3,4,5])).send(bob)

In [33]:
x

Variable containing:FloatTensor[_PointerTensor - id:77495490293 owner:me loc:bob id@loc:95460224931]

In [34]:
bob._objects

{76564227868: [_LocalTensor - id:76564227868 owner:bob],
 95460224931: [_LocalTensor - id:95460224931 owner:bob],
 84274628672: [_LocalTensor - id:84274628672 owner:bob],
 38525937495: [_LocalTensor - id:38525937495 owner:bob]}

In [35]:
alice._objects

{}

In [36]:
x = x.move(alice)

In [37]:
bob._objects

{}

In [38]:
alice._objects

{50436060125: [_LocalTensor - id:50436060125 owner:alice],
 51028287458: [_PointerTensor - id:51028287458 owner:alice loc:bob id@loc:84274628672],
 78005829810: [_PointerTensor - id:78005829810 owner:alice loc:bob id@loc:38525937495],
 62807694469: [_LocalTensor - id:62807694469 owner:alice],
 84274628672: [_LocalTensor - id:84274628672 owner:alice],
 38525937495: [_LocalTensor - id:38525937495 owner:alice]}

In [39]:
x

Variable containing:FloatTensor[_PointerTensor - id:40512548360 owner:me loc:alice id@loc:50436060125]